In [ ]:
# Runs multiple calculations on a database of atoms objects on Warwick's Orac and Tinis clusters. 

# SSH

In [73]:
from paramiko import SSHClient,config
ssh = SSHClient()
ssh.load_system_host_keys()
max_jobs = 15
user = 'msrvhs'
base_dir = '/home/chem/msrvhs/work/e_loss/p4x4_co_cu100/'
base_local_dir = 'e_loss/'

def qcheck(cluster):
    #returns false if user has 15 or more jobs in cluster queue. True otherwise
    if cluster == 'tinis':
        ssh.connect('tinis.csc.warwick.ac.uk',username=user)
    elif cluster == 'orac':
        ssh.connect('orac.csc.warwick.ac.uk',username=user)
    if cluster == 'orac' or cluster == 'tinis':
        stdin, stdout, stderr  = ssh.exec_command("squeue -u "+user+" -o %t")
        if (len((stdout.channel.recv(1024)).split('\n'))-2) < max_jobs:
            return True
        if (len((stdout.channel.recv(1024)).split('\n'))-2) >= max_jobs:
            return False
        else:
            print('Cannot read cluster queue!')
        ssh.close()
    
def submit_calc(row,calc):
    import time
    #attaches specified calculator to specified atoms object. Creates control.in, geometry.in and submit script
    #for specified cluster and sends to that cluster and submits the job
    idir = str(row.id)+'/'
    
    #make local subdirectory if doesnt exist
    if not os.path.exists(base_local_dir+idir):
        os.mkdir(base_local_dir+idir)
    
    #write geometry and control
    atoms = con.get_atoms(id=row.id)
    with cd(base_local_dir+idir):
        if row.calculation_status == 0:
            calc1.write_input(atoms)
        if row.calculation_status == 1:
            calc2.write_input(atoms)
        if row.calculation_status == 2:
            calc3.write_input(atoms)
            
    #make submit file
    submit_file = 'submit.sh'
    with open(base_local_dir+idir+submit_file, 'w+') as f:
        if row.cluster == 'tinis':
            text = tinis_submit.replace('ID',str(row.id))
            text = text.replace('JOBNAME',str(row.calculation_status))
            f.write(text)
        elif row.cluster == 'orac':
            text = orac_submit.replace('ID',str(row.id))
            text = text.replace('JOBNAME',str(row.calculation_status))
            f.write(text)
    
    #add command to submit to move results to file
    with open(base_local_dir+idir+submit_file, "a") as f:
        f.write('\nmv friction_tensor.out friction_tensor_'+str(row.id)+'_'+str(row.calculation_status)+'.out')
    
    #clean up if last calculation
    if row.calculation_status == 2:
        with open(submit_file, "a") as f:
            f.write("\nrm first_order_*")
            
    #connect to cluster
    if row.cluster == 'tinis':
        ssh.connect('tinis.csc.warwick.ac.uk',username=user)
    elif row.cluster == 'orac':
        ssh.connect('orac.csc.warwick.ac.uk',username=user)
        
    #send directory to cluster if status =0, else send contents    
    if row.cluster == 'orac' or row.cluster == 'tinis':
        if row.calculation_status == 0:  
            ssh.exec_command('cd '+base_dir+' ;mkdir '+idir)
            time.sleep(2)
            
        ftp_client=ssh.open_sftp()
        ftp_client.put(base_local_dir+idir+'control.in',base_dir+idir+'control.in')
        ftp_client.put(base_local_dir+idir+'geometry.in',base_dir+idir+'geometry.in')
        ftp_client.put(base_local_dir+idir+submit_file,base_dir+idir+submit_file)
            
        ftp_client.close()
        
        #submit job
        ssh.exec_command('cd '+base_dir+idir+' ;sbatch '+submit_file)
        con.update(row.id, calculation_status=row.calculation_status+1)
        
def check_results(row):
    idir = str(row.id)+'/'
    #Check if results are present, if so then read in and update calculation status. 
    #If not then do nothing
    
    #ssh
    if row.cluster == 'tinis':
        ssh.connect('tinis.csc.warwick.ac.uk',username=user)
    elif row.cluster == 'orac':
        ssh.connect('orac.csc.warwick.ac.uk',username=user)
    #if friction_tensor_X exists where X is the calculation -1, as added 1 at end of submit, return true
    if row.cluster == 'orac' or row.cluster == 'tinis':
        ftp_client=ssh.open_sftp()
        return (sftp_exists(ftp_client,base_dir+idir+'friction_tensor_'+str(row.id)+'_'+str(row.calculation_status-1)+'.out'))
        ftp_client.close()
    else:
        print('Entry has no attached cluster!')
            
        
def get_results(row):
    idir = str(row.id)+'/'
        #ssh
    if row.cluster == 'tinis':
        ssh.connect('tinis.csc.warwick.ac.uk',username=user)
    elif row.cluster == 'orac':
        ssh.connect('orac.csc.warwick.ac.uk',username=user)
        
    #copy friction_tensor_X and aims.out to local directory
    if row.cluster == 'orac' or row.cluster == 'tinis':
        
        remote_ft = base_dir+idir+'friction_tensor_'+str(row.id)+'_'+str(row.calculation_status-1)+'.out'
        remote_out = base_dir+idir+'_'+str(row.calculation_status-1)+'.aims.out'
        local_path = base_local_dir+idir
        
        ftp_client=ssh.open_sftp()
        ftp_client.get(remote_ft,local_path)
        ftp_client.get(remote_out,local_path)
        ftp_client.close()
        
def read_results(row):
    #Parse the friction tensor from aims.out into database ft1, ft2 or ft3
    local_path = base_local_dir+idir+'_'+str(row.calculation_status-1)+'.aims.out'
    #Reads in tensor from aims.out. Made specifically for large tensors where each row of the tensor is
#printed over multiple lines in aims.out but should work in normal (small) tensors. With a geometry.in file present
#it labels the tensor with the atoms and the corresponding .csv file. No arguments needed

    friction_tensor = read_tensor_from_aims(local_path)
    if row.calculation_status == 1:   
        con.update(row.id,ft1=friction_tensor)
    elif row.calculation_status == 2:
        con.update(row.id,ft2=friction_tensor)
    elif row.calculation_status == 3: 
        con.update(row.id,ft3=friction_tensor)


def read_tensor_from_aims(path):
    import numpy as np


    #Reading tensor from aims.out
    lines = open(path).readlines()
    raw_lines = []
    copy = False
    for line in lines:
        if '********Printing Friction Tensor in 1/ps*********' in line:
            copy = True
            continue
        if '**********END Printing Friction Tensor************' in line:
            copy = False
        if copy:
            raw_lines.append(line)
            #for j in range(ndim):
                #friction_tensor[i,j]=float(line.split()[j])
            
    a = len(raw_lines[0].split())
    lines_per_line = 0
    for i,line in enumerate(raw_lines):
        if i == 0:
            continue
        if len(line.split()) != a:
            lines_per_line += 1
        if len(line.split()) == a:
            break

    ndim = (a - 1)*(lines_per_line+1)
    print(ndim)
    friction_tensor = np.zeros((ndim,ndim))
    print('lines per line '+str(lines_per_line+1))
    c=0
    i=-1
    for ii, line in enumerate(raw_lines):
    
        if c == lines_per_line+1 or c==0:
            i+=1
            c=0
    
        for j in range(ndim/(lines_per_line+1)):
            if c ==0:
                friction_tensor[i,j]=float(line.split()[j+1])
            else:
                friction_tensor[i,j+((ndim/(lines_per_line+1))*c)]=float(line.split()[j])
        c+=1
        
    return(friction_tensor)
    
    
def sftp_exists(sftp, path):
    #check if file exists on remote host from 
    #https://stackoverflow.com/questions/850749/check-whether-a-path-exists-on-a-remote-host-using-paramiko/1391409
    """os.path.exists for paramiko's SCP object
    """
    import errno
    try:
        sftp.stat(path)
    except IOError, e:
        if e.errno == errno.ENOENT:
            return False
        raise
    else:
        return True
    
import os

class cd:
    """Context manager for changing the current working directory"""
    """https://stackoverflow.com/questions/431684/how-do-i-change-directory-cd-in-python/13197763#13197763"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [74]:
def cluster_matrices_use(database):
    #Run through the database, check how many currently have friction_matrices outputted on clusters. 
    #Returns false for each cluster if number above a set maxmium value for cluster
    #TODO: additionally ssh into cluster and check the actual usage vs quota
    o=0
    t=0
    orac_max_matrices = 5
    tinis_max_matrices = 15
    orac_space = True
    tinis_space = True
    
    for i in range(1,len(database)+1):
        row = con.get(id=i)
        if hasattr(row, 'cluster'):
            if row.cluster == 'orac':
                if row.calculation_status > 0 and row.calculation_status < 4:
                    o +=1
            if row.cluster == 'tinis':
                if row.calculation_status > 0 and row.calculation_status < 4:
                    t += 1
        else:
            continue
            
    if o >= orac_max_matrices:
        orac_space=False
    if t >= tinis_max_matrices:
        tinis_space = False
        
    return(orac_space,tinis_space)

# Calculators

In [36]:
friction_atom_indices = [64,65]
species_dir = '/home/chem/msrvhs/software/aims/fhiaims/fhiaims/species_defaults/light/'
calc1 = Aims(
        xc='pbe',
        tier=[1,2,2], #light with second tier adsorbates
        output=['k_point_list','friction_matrices'],#,'friction_eigenvectors'],
        sc_accuracy_etot = 1e-6,
        sc_accuracy_rho = 1e-5,
        sc_accuracy_eev = 0.001,
#        sc_accuracy_forces = 0.001,
        sc_iter_limit = 100,
        occupation_type = ['gaussian', 0.1],
        relativistic = ['atomic_zora','scalar'],
        spin = 'none',
        empty_states = 100,
        calculate_friction = 'numerical_friction',
        friction_numeric_disp = 0.0025,
        friction_broadening_width = 0.6,
        friction_iter_limit = 50, 
        friction_temperature = 300,
        k_grid = [13,13,1],
        species_dir = species_dir,
        friction_atoms = friction_atom_indices,
        restart_read_only ='wvfn.dat'
        )

calc2 = Aims(
        xc='pbe',
        tier=[1,2,2], #light with second tier adsorbates
        output=['k_point_list'],#,'friction_eigenvectors'],
        sc_accuracy_etot = 1e-6,
        sc_accuracy_rho = 1e-5,
        sc_accuracy_eev = 0.001,
#        sc_accuracy_forces = 0.001,
        sc_iter_limit = 100,
        occupation_type = ['gaussian', 0.1],
        relativistic = ['atomic_zora','scalar'],
        spin = 'none',
        empty_states = 100,
        calculate_friction = 'numerical_friction',
        friction_numeric_disp = 0.0025,
        friction_broadening_width = 0.1,
        friction_iter_limit = 50, 
        friction_temperature = 300,
        k_grid = [13,13,1],
        species_dir = species_dir,
        friction_atoms = friction_atom_indices,
        friction_read_matrices = '.true.',
        restart_read_only ='wvfn.dat'
        )

calc3 = Aims(
        xc='pbe',
        tier=[1,2,2], #light with second tier adsorbates
        output=['k_point_list'],#,'friction_eigenvectors'],
        sc_accuracy_etot = 1e-6,
        sc_accuracy_rho = 1e-5,
        sc_accuracy_eev = 0.001,
#        sc_accuracy_forces = 0.001,
        sc_iter_limit = 100,
        occupation_type = ['gaussian', 0.1],
        relativistic = ['atomic_zora','scalar'],
        spin = 'none',
        empty_states = 100,
        calculate_friction = 'numerical_friction',
        friction_numeric_disp = 0.0025,
        friction_broadening_width = 0.01,
        friction_iter_limit = 50, 
        friction_temperature = 300,
        k_grid = [13,13,1],
        species_dir = species_dir,
        friction_atoms = friction_atom_indices,
        friction_read_matrices = '.true.',
        restart_read_only ='wvfn.dat'
        )


# Submit scripts

In [8]:
orac_submit = """#!/bin/bash \n
#SBATCH -J ID_JOBNAME \n
#SBATCH --nodes=4 \n
#SBATCH --ntasks-per-node=28 \n
#SBATCH --cpus-per-task=1 \n
#SBATCH --mem-per-cpu=4571 \n
#SBATCH --time=48:00:00 \n
#SBATCH --mail-type=FAIL \n
#SBATCH --mail-user=Connor.Box@warwick.ac.uk \n
\n
ulimit -s unlimited \n
module load intel/2018.1.163-GCC-6.4.0-2.28 \n
module load impi/2018.1.163 \n
module load imkl/2018.1.163 \n
\n
export OMP_NUM_THREADS=1 \n
export MKL_NUM_THREADS=1 \n
export MKL_DYNAMIC=FALSE \n
\n
MY_NUM_THREADS=$SLURM_CPUS_PER_TASK \n
\n
\n
AIMS_BIN=/home/chem/msrvhs/software/aims/libxc_may19/bin/aims.190121.mpi.x \n
cp /home/chem/msrvhs/work/e_loss/p4x4_co_cu100/scf/wvfn.dat* . \n
srun $AIMS_BIN > ID_JOBNAME.aims.out \n
rm wvfn.dat* """

tinis_submit = """#!/bin/bash \n
#SBATCH -J ID_JOBNAME \n
#SBATCH --nodes=4 \n
#SBATCH --ntasks-per-node=16 \n
#SBATCH --cpus-per-task=1 \n
#SBATCH --mem-per-cpu=3882 \n
#SBATCH --time=48:00:00 \n
#SBATCH --mail-type=FAIL \n
#SBATCH --mail-user=Connor.Box@warwick.ac.uk \n
\n
ulimit -s unlimited \n
module load intel/2018.1.163-GCC-6.4.0-2.28 \n
module load impi/2018.1.163 \n
module load imkl/2018.1.163 \n
\n
export OMP_NUM_THREADS=1 \n
export MKL_NUM_THREADS=1 \n
export MKL_DYNAMIC=FALSE \n
\n
MY_NUM_THREADS=$SLURM_CPUS_PER_TASK \n
\n
\n
AIMS_BIN=/home/chem/msrvhs/software/aims/libxc_may19/bin/aims.190121.mpi.x \n
cp /home/chem/msrvhs/work/e_loss/p4x4_co_cu100/scf/wvfn.dat* . \n
srun $AIMS_BIN > ID_JOBNAME.aims.out \n
rm wvfn.dat* """

# Database

In [22]:
from ase.io import read,write
from ase.calculators.aims import Aims
from ase.visualize import view
import numpy as np
from ase import Atoms
import numpy as np
import os
from ase.db import connect

In [23]:
#atoms = []
#for i in np.arange(45,54,1):
#    atoms += read('{0:02d}/aims.out'.format(i),':')

#print('read in')

read in


In [68]:
#con = connect('database.db')
#for a in atoms:
#    con.write(a,calculation_status=0)

TypeError: 'SQLite3Database' object is not callable

# Job submit

In [75]:
if qcheck('tinis')==False and qcheck('orac')==False:
    #quit()
    print('Clusters full')
    
if not os.path.exists(base_local_dir):
        os.mkdir(base_local_dir)

In [76]:
for i in range(1,16):
    print('Calculating entry ', i)
    row = con.get(id=i)   
    if row.calculation_status == 0: #check if cluster available then submit1
        orac_space,tinis_space = cluster_matrices_use(con)
        if tinis_space:
            if qcheck('tinis'):
                con.update(i, cluster='tinis')
                row = con.get(id=i)
                submit_calc(row,calc1)
                print('submitted a calc1 on tinis')
        elif orac_space:
            if qcheck('orac'):
                con.update(i, cluster='orac')
                row = con.get(id=i)
                submit_calc(row,calc1)
                print('submitted a calc1 on orac')
                
    if row.calculation_status == 1: #checkjob #if done then do next
        if qcheck(row.cluster):
            if check_results(row):
                submit_calc(row,calc2)
                print('submitted a calc2 on '+row.cluster)
            
    if row.calculation_status == 2: #checkjob if done do next
        if qcheck(row.cluster):
            if check_results(row):
                read_results(row)
                submit_calc(row,calc3)
                print('submitted a calc3 on '+row.cluster)
            
    if row.calculation_status == 3: #checkjob if done do cleanup
        if qcheck(row.cluster):
            if check_results(row):
                read_results(row)
                con.update(i, calculation_status=4)
            
    if row.calculation_status == 4:
        continue
        

('Calculating entry ', 1)
('Calculating entry ', 2)
('Calculating entry ', 3)
('Calculating entry ', 4)
('Calculating entry ', 5)
('Calculating entry ', 6)
('Calculating entry ', 7)
('Calculating entry ', 8)
('Calculating entry ', 9)
('Calculating entry ', 10)
('Calculating entry ', 11)
('Calculating entry ', 12)
('Calculating entry ', 13)
submitted a calc1 on tinis
('Calculating entry ', 14)
submitted a calc1 on tinis
('Calculating entry ', 15)
submitted a calc1 on tinis
